In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import Model, Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score 

In [2]:
!wget -O creditfraud.zip https://www.dropbox.com/s/tl20yp9bcl56oxt/creditcardfraud.zip?dl=0

--2022-11-14 12:45:27--  https://www.dropbox.com/s/tl20yp9bcl56oxt/creditcardfraud.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/tl20yp9bcl56oxt/creditcardfraud.zip [following]
--2022-11-14 12:45:27--  https://www.dropbox.com/s/raw/tl20yp9bcl56oxt/creditcardfraud.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2e028256ece895b0498935b038.dl.dropboxusercontent.com/cd/0/inline/BwtXwUWnlfFQF1WaJ5bayVNApJyhymPPda0h8bXO4IZmjP-eWBrJ-U9Oa3busOxfGso4y1yjlQacG5Tl5OfnwYrEhjSgkmALB7QtJn3AMr3q97MTDSewAgy-GpkALx42r3XfEqf4l14amZn5TjMHvPfQpnkvwKRyxsxQeVeb9nq45Q/file# [following]
--2022-11-14 12:45:27--  https://uc2e028256ece895b0498935b038.dl.dropboxusercontent.com/cd/0/inline/BwtXwUWnlfFQF1WaJ5bayVNApJyhymPPda0h8bXO4IZmjP-eWBrJ-U

In [3]:
!unzip creditfraud.zip

Archive:  creditfraud.zip
  inflating: creditcard.csv          


In [4]:
raw_data = pd.read_csv("/content/creditcard.csv")

In [5]:
raw_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [19]:
raw_data = raw_data.drop('Time', axis = 1) # irrelevant feature

In [20]:
labels = raw_data.iloc[:, -1]
data = raw_data.iloc[:, : -1]

data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [21]:
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [22]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2)

In [23]:
y_train.value_counts()


0    227460
1       385
Name: Class, dtype: int64

In [24]:
normal_data_train = x_train[y_train == 0]
normal_data_test = x_test[y_test == 0]
anomalous_data_train = x_train[y_train == 1]
anomalous_data_test = x_test[y_test == 1]

In [25]:
normal_data_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
191121,1.853492,-0.165644,-1.905759,0.112514,1.136156,1.100457,-0.156047,0.382927,0.277170,-0.356811,...,-0.201031,-0.224436,-0.541216,0.373528,-0.759148,-0.513898,0.262404,-0.020577,-0.049671,29.99
222846,-0.570405,1.003528,0.732925,-1.030464,0.332070,-0.058928,0.308654,0.312099,0.450547,-0.870506,...,0.198572,-0.257265,-0.550060,0.003387,0.563773,-0.140820,-0.299126,0.391257,0.162629,7.40
105407,-1.518039,0.361303,2.194009,-0.493950,-0.604042,1.598940,-0.927008,-0.273059,0.256210,0.678892,...,-0.514393,0.260765,-0.660586,-0.263521,-0.857898,0.329874,-0.370340,0.319318,0.279850,20.00
186128,1.935311,-1.125802,-2.064290,-2.140113,1.498755,3.415246,-1.175147,0.922250,1.672035,-0.624354,...,0.020295,0.226618,0.625685,0.142804,0.767508,-0.219218,0.327496,0.002688,-0.044586,62.41
278222,-4.424236,-0.790861,-0.165574,0.343182,-1.728279,-0.264672,0.480408,0.800980,0.013922,-1.244242,...,-1.644620,-0.399914,-0.387785,0.289101,0.041093,0.420951,-0.394134,0.051731,-1.378731,169.05


In [36]:
# define model
class AnomalyDetector(Model):
  def __init__(self, output_units):
    super().__init__()

    self.encoder = Sequential([
        tfl.Dense(16, activation = 'relu'),
        tfl.Dense(8, activation = 'relu')
    ])

    self.decoder = Sequential([
        tfl.Dense(16, activation = 'relu'),
        tfl.Dense(output_units, activation = 'sigmoid')
    ])

  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded

In [37]:
model =  AnomalyDetector(29)

In [38]:
model.compile(optimizer="adam", loss = "mse")

In [39]:
model.fit(normal_data_train, normal_data_train, epochs = 20, batch_size = 128)

Epoch 1/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.4893
Epoch 2/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3911
Epoch 3/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3740
Epoch 4/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3672
Epoch 5/20
1778/1778 [==============================] - 3s 2ms/step - loss: 2457.3633
Epoch 6/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3643
Epoch 7/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3594
Epoch 8/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3591
Epoch 9/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3574
Epoch 10/20
1778/1778 [==============================] - 4s 2ms/step - loss: 2457.3555
Epoch 11/20
1778/1778 [==============================] - 3s 2ms/step - loss: 2457.3530
Epoch 12/20
1778/1778 [=============================

In [40]:
# define threshold
reconstructions = model.predict(normal_data_train)
loss = tf.keras.losses.mse(reconstructions, normal_data_train)
threshold = np.mean(loss) + np.std(loss)

7109/7109 [==============================] - 9s 1ms/step


In [41]:
# detect anomaly
reconstructions_test = model.predict(x_test)
loss = tf.keras.losses.mse(reconstructions_test, x_test)
predictions = tf.math.less(loss, threshold)

1781/1781 [==============================] - 2s 1ms/step


In [43]:
#accuracy
accuracy = accuracy_score(y_test, predictions)
print(accuracy * 100)

0.7513781117236052
